### Library Imports

In [97]:
import pandas as pd
import xarray as xr
import numpy as np

### Data imports

In [107]:
# Data info : 
nb_samples = 4 # 4 samples per year
filename = 'France_mix_test.xlsx'
#filename = 'elect_table.xlsx'

In [108]:
elec_sheets = {}

for sample in range(nb_samples) : 
    year_day_sample = pd.DataFrame(pd.read_excel(filename,index_col = 0,sheet_name = sample))
    # Save, from the get-go, the possible years (needs more work to be fail proof)
    years = year_day_sample['YEAR'].unique()
    day_samples = year_day_sample.groupby('YEAR')
    
    elec_sheets[sample] = {}

    # Split years from the sheets    
    for year,data in day_samples : 
        data.drop(columns = 'YEAR',inplace = True)
        # Store both years according to number of samples : 
        elec_sheets[sample][year] =data

#print('Samples within a year :',elec_sheets.keys())
#print('Sample of years ',elec_sheets[0].keys())    
#print(years)    
    #elec_sheets[sample] = pd.DataFrame(pd.read_excel(filename,index_col = 0,sheet_name = sample))
display(elec_sheets[0][2020].head(5))

#ds_W_2020 = pd.read_excel('elect_table.xlsx', index_col = 'Hr')
#display(ds_2020.head(5))

Samples within a year : dict_keys([0, 1, 2, 3])
Sample of years  dict_keys([2020, 2050])


,Nucleaire,Eolien en mer,Eolien terrestre,Centrales gaz,Solaire,STEP,Hydro,Imports nets,Autre renouvelables,Batteries
Hours,,,,,,,,,,
0,5.0,30.0,0.1,1.0,0.1,0.1,0.1,0.1,0.1,0.1
1,5.0,30.0,0.1,2.0,0.1,0.1,0.1,0.1,0.1,0.1
2,5.0,30.0,0.1,3.0,0.1,0.1,0.1,0.1,0.1,0.1
3,5.0,30.0,0.1,4.0,0.1,0.1,0.1,0.1,0.1,0.1
4,5.0,30.0,0.1,5.0,0.1,0.1,0.1,0.1,0.1,0.1


In [109]:
# List the dimensions
Hours =  elec_sheets[0][years[0]].index.tolist()

# Technologies : 
cols2keep = []
for sheet_id in elec_sheets.keys() :
    for year in years : # redundant, but usefull to make sure nothing is off according to years
        cols = elec_sheets[sheet_id][years[0]].columns.to_list()   
        cols2keep.extend(cols)
technologies = [x for x in set(cols2keep)]

# Adapt dataframes in case some technologies are missing at this timepoint : 
for sheet_id in elec_sheets.keys() :
    for year in years : 
        # Find missing columns
        cols_to_add = [x for x in technologies if x not in elec_sheets[sheet_id][year].columns]

        # Add the columns (if needed) : 
        for col in cols_to_add :
            print(col)
            elec_sheets[sheet_id][year][col] = 0

        # While there, also make sure if a column is partial, just add zeros
        elec_sheets[sheet_id][year] = elec_sheets[sheet_id][year].fillna(0)

        # And also : relabel according to the technologies list (to match when the xarray is built)
        elec_sheets[sheet_id][year]= elec_sheets[sheet_id][year][technologies]
        
        # Quick check (optional)
        #print(year)
        #display(elec_sheets[sheet_id][year].head(5))

# Number of snapshots available for the year
date2_year = [0]
for value in range(nb_samples): 
    date2_year.append(round((value+1)*365/nb_samples))

# Overview
print(Hours)
print(technologies)
print(date2_year)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
['Eolien en mer', 'Autre renouvelables', 'Nucleaire', 'Hydro', 'Centrales gaz', 'Imports nets', 'Batteries', 'Eolien terrestre', 'STEP', 'Solaire']
[0, 91, 182, 274, 365]


In [110]:
# list the dataframes that need to be uploaded to xarray
yearly_data = []

# First by year
for year in years : 
    # Then by sample number (season here)
    data = [elec_sheets[sheet_id][year] for sheet_id in range(nb_samples)]
    
    # Append at the end the first entry (coming full circle) 
    data.append(elec_sheets[0][year])
    yearly_data.append(data)

In [114]:
# Creating the xarray DataArray
elec_mixes = xr.DataArray(
    yearly_data,
    coords=[years,date2_year,Hours, technologies],
    dims=["year","Date", "Hour","technology"]
)

In [115]:
# Check if its is properly working : 
display(elec_mixes.sel(
    #technology='Centrales gaz', 
    year = 2020,
    Date = 365,
    Hour = 3,
))

<xarray.DataArray (technology: 10)>
array([30. ,  0.1,  5. ,  0.1,  4. ,  0.1,  0.1,  0.1,  0.1,  0.1])
Coordinates:
    year        int64 2020
    Date        int32 365
    Hour        int32 3
  * technology  (technology) <U19 'Eolien en mer' ... 'Solaire'

In [120]:
# Export for next steps.
elec_mixes.to_netcdf(path = 'elec_mixes.nc')

In [124]:
# Reload and make sure everything is okay : 
#elec_mixes_test = xr.open_dataset('elec_mixes.nc')
#display(elec_mixes)